<div style="padding:20px 30px 30px; 
            color:#004346;
            font-size:40px;
            display:fill;
            text-align:center;
            border-radius:20px;
            border: 5px double;
            border-color:#9966FF;
            background-color: #FFFFCC;
            overflow:hidden;
            font-weight:400"> 
<p style="font-weight: bold; text-align: center;"> Project:  English Score </p>
    
</div>

<div style="padding:0px 40px 30px; 
            color:#004346;
            font-size:110%;
            display:fill;
            border-radius:20px;
            border: 5px double;
            border-color:#9966FF;
            background-color: #FFFFCC;
            overflow:hidden;
            font-weight:450;"> 
    
__Постановка проблемы:__ Просмотр фильмов на оригинальном языке - это популярный и действенный метод прокачаться при изучении иностранных языков. Важно выбрать фильм, который подходит студенту по уровню сложности, т.е. студент понимал 50-70 % диалогов. Чтобы выполнить это условие, преподаватель должен посмотреть фильм и решить, какому уровню он соответствует. Однако это требует больших временных затрат.
    
__Цель:__ Разработать ML решение для автоматического определения уровня сложности англоязычных фильмов, разработать для неё веб-интерфейс и создать микросервис. 
    
__Описание данных:__

- субтитры фильмов, сохраненные в директориях, названия которых, соответствуют уровню сложности по шкале CEFR([Common European Framework of Reference]
    
- субтитры фильмов и [фaйл xlsx], содержаший название фильмов и уровню сложности по шкале CEFR
    
- список слов, по уровлю сложности Oxford level.
</div>        

In [1]:
!pip install pysrt

In [2]:
!pip install optuna

In [3]:
!pip install spacy
!python -m spacy download en_core_web_sm

                                              0.0/12.8 MB ? eta -:--:--
                                              0.0/12.8 MB 1.3 MB/s eta 0:00:10
                                              0.0/12.8 MB 1.3 MB/s eta 0:00:10
                                             0.1/12.8 MB 544.7 kB/s eta 0:00:24
                                             0.1/12.8 MB 737.3 kB/s eta 0:00:18
                                             0.2/12.8 MB 893.0 kB/s eta 0:00:15
                                              0.2/12.8 MB 1.1 MB/s eta 0:00:12
                                              0.3/12.8 MB 1.1 MB/s eta 0:00:12
     -                                        0.4/12.8 MB 1.2 MB/s eta 0:00:11
     -                                        0.4/12.8 MB 1.2 MB/s eta 0:00:11
     -                                        0.4/12.8 MB 1.2 MB/s eta 0:00:11
     -                                        0.6/12.8 MB 1.5 MB/s eta 0:00:08
     --                                       0.7/12.8 M

In [4]:
!pip install xgboost

In [5]:
!pip install lightgbm

In [6]:
!pip install catboost

<div style="padding:0px 20px 10px; 
            color:#004346;
            font-size:15px;
            display:fill;
            text-align:center;
            border-radius:20px;
            border: 5px double;
            border-color:#9966FF;
            background-color: #FFFFCC;
            overflow:hidden;
            font-weight:400"> 

# Используемые библиотеки

</div>

In [84]:
from pathlib import Path
import pysrt
import pandas as pd
import string
import spacy
import csv
import numpy as np
import nltk
nltk.download('omw-1.4')
from nltk.corpus import stopwords
from  nltk.tokenize import word_tokenize
import joblib
import re
import random
import optuna
import warnings

from nltk.stem.porter import PorterStemmer
porter_stemmer = PorterStemmer()

from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack
from sklearn.model_selection import train_test_split,GridSearchCV,RandomizedSearchCV
from sklearn.metrics import make_scorer, mean_absolute_error
from sklearn.pipeline import Pipeline
from sklearn.utils.class_weight import compute_sample_weight, compute_class_weight
from sklearn.ensemble import RandomForestRegressor 

from catboost import Pool, CatBoostRegressor

# константы
RANDOM_SEED = 3826
EARLY_STOPPING_ROUND = 100
CV = 3
VERBOSE = 100

HTML = r'<.*?>' # html тэги меняем на пробел
TAG = r'{.*?}' # тэги меняем на пробел
COMMENTS = r'[\(\[][A-Za-z ]+[\)\]]' # комменты в скобках меняем на пробел
UPPER = r'[[A-Za-z ]+[\:\]]' # указания на того кто говорит (BOBBY:)
LETTERS = r'[^a-zA-Z\'.,!? ]' # все что не буквы меняем на пробел 
SPACES = r'([ ])\1+' # повторяющиеся пробелы меняем на один пробел
DOTS = r'[\.]+' # многоточие меняем на точку
SYMB = r"[^\w\d'\s]" # знаки препинания кроме апострофа

# настройки блокнота
warnings.filterwarnings('ignore')

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\ratim\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


<div style="padding:0px 20px 10px; 
            color:#004346;
            font-size:15px;
            display:fill;
            text-align:center;
            border-radius:20px;
            border: 5px double;
            border-color:#9966FF;
            background-color: #FFFFCC;
            overflow:hidden;
            font-weight:400"> 

# Загрузка и первичная обработка данных

</div>

In [9]:
try:
    # путь к папке с файлами субтитров
    subtitles_folder = Path('project_eng_scr/English_scores/Subtitles_all/level')
    subtitles_folder2 = 'project_eng_scr/English_scores/Subtitles_all/Subtitles'
    excel_file = 'project_eng_scr/English_scores/movies_labels.xlsx'
    # загрузка данных для рассчета статистической информации:
    a1_list = next(csv.reader(open('project_eng_scr/English_scores/a1.csv', 'r')))
    a2_list = next(csv.reader(open('project_eng_scr/English_scores/a2.csv', 'r')))
    b1_list = next(csv.reader(open('project_eng_scr/English_scores/b1.csv', 'r')))
    b2_list = next(csv.reader(open('project_eng_scr/English_scores/b2.csv', 'r')))
    c1_list = next(csv.reader(open('project_eng_scr/English_scores/c1.csv', 'r')))
    display('Все данные загруженны коректно')
except:
    display('Данные не доступны')

'Все данные загруженны коректно'

In [10]:
#проверяем как загрузизись данные
print(a1_list[:5])
print(a2_list[:5])
print(b1_list[:5])
print(b2_list[:5])
print(c1_list[:5])
exel = pd.read_excel(excel_file)
display(exel.head())

['bored', 'later', 'other', 'colour', 'friday']
['engineer', 'jump', 'later', 'death', 'shape']
['divide', 'battery', 'killing', 'studio', 'chest']
['holy', 'popularity', 'committee', 'nasty', 'rescue']
['morality', 'superb', 'acquisition', 'arbitrary', 'earnings']


,id,Movie,Level
0,0,10_Cloverfield_lane(2016),B1
1,1,10_things_I_hate_about_you(1999),B1
2,2,A_knights_tale(2001),B2
3,3,A_star_is_born(2018),B2
4,4,Aladdin(1992),A2/A2+


In [11]:
# функция для первичной обработки текста
def clean_subs(txt):
    txt = re.sub(HTML, ' ', subs.text) # html тэги меняем на пробел
    txt = re.sub(COMMENTS, ' ', txt) # комменты в скобках меняем на пробел
    txt = re.sub(UPPER, ' ', txt) # указания на того кто говорит (BOBBY:)
    txt = re.sub(LETTERS, ' ', txt) # все что не буквы меняем на пробел
    txt = re.sub(DOTS, r'.', txt) # многоточие меняем на точку
    txt = re.sub(SPACES, r'\1', txt) # повторяющиеся пробелы меняем на один пробел
    txt = re.sub(SYMB, '', txt) # знаки препинания кроме апострофа на пустую строку
    txt = re.sub('www', '', txt) # кое-где остаётся www, то же меняем на пустую строку
    txt = txt.lstrip() # обрезка пробелов слева
    txt = txt.encode('ascii', 'ignore').decode() # удаляем все что не ascii символы   
    txt = txt.lower() # текст в нижний регистр
    return txt

### Получение первой части размеченных данных

In [12]:
# получение списока файлов субтитров в папке
subtitles_files = subtitles_folder.rglob('*.srt')
# cоздание пустого датафрейма для хранения данных
df = pd.DataFrame(columns=['movie', 'subtitles', 'label'])
# загрузка и первичня обработка субтитров
data = []
for file_path in subtitles_files:
    subs = pysrt.open(str(file_path), encoding='latin-1')
    if len(subs) == 0:
        subs = pysrt.open(str(file_path), encoding='utf-16') #учитываем альтернативную кодировку
    txt = ' '.join([sub.text for sub in subs])
    subtitle_text = clean_subs(txt)
    data.append({'movie': file_path.name[:-4], 'subtitles': subtitle_text, 'label': file_path.parent.name})
    
# объединяем все записи в датасете с помощью функции concat
df = pd.concat([df, pd.DataFrame(data)], ignore_index=True)

In [13]:
df.head()

,movie,subtitles,label
0,The Walking Dead-S01E01-Days Gone Bye.English,little girl i'm a policeman little girl don't ...,A2
1,The Walking Dead-S01E02-Guts.English,mom right here any luck how do we tell if they...,A2
2,The Walking Dead-S01E03-Tell It To The Frogs.E...,that's right you heard me bitch you got a prob...,A2
3,The Walking Dead-S01E04-Vatos.English,what nothing it's not nothing it's always some...,A2
4,The Walking Dead-S01E05-Wildfire.English,walkie talkie squawks morgan i don't know if y...,A2


In [14]:
df.shape

(163, 3)

In [15]:
df['label'].unique()

array(['A2', 'B1', 'B2', 'C1'], dtype=object)

### Получение второй части размеченных данных

In [16]:
# загрузка датафрейма из файла excel
df2 = pd.read_excel(excel_file)
# удаление столбца, не использующего в дальнейшем анализе: `id`
df2 = df2.drop('id', axis=1)
# переименование признаков для последующей конкатинации датасетов
df2.rename(columns = {
    'Movie':'movie',
    'Level':'label'
    }, inplace = True )

In [17]:
# функция для добавления судтитров, по названию фильмов и первичной обработки
def add_srt(x):
    try:
        file_path = subtitles_folder2+x+'.srt'
        subs = pysrt.open(file_path, encoding='latin-1')
        if len(subs) == 0:
            subs = pysrt.open(str(file_path), encoding='utf-16') #учитываем альтернативную кодировку
        txt = ' '.join([sub.text for sub in subs])
        clean_text = clean_subs(txt)
        return clean_text
    except FileNotFoundError:
        return np.nan

In [18]:
df2['label'].unique()

array(['B1', 'B2', 'A2/A2+', 'C1', 'B1, B2', 'A2/A2+, B1', 'A2'],
      dtype=object)

In [19]:
#приводим значения A2/A2+ к стандартному значению B1 
list_b1= df2.query('label == "A2/A2+, B1"').index.to_list()
for i in list_b1:
    df2.loc[i,'label']='B1'

list_a2= df2.query('label == "A2/A2+"').index.to_list()
for r in list_a2:
    df2.loc[r,'label']='B1'
#приводим значения B1, B2 к стандартному значению B1
list_b2= df2.query('label == "B1, B2"').index.to_list()
for q in list_b2:
    df2.loc[q,'label']='B1'

In [20]:
df2['label'].unique()

array(['B1', 'B2', 'C1', 'A2'], dtype=object)

In [21]:
df2['subtitles'] = df2['movie'].apply(add_srt)

In [22]:
df2.head()

,movie,label,subtitles
0,10_Cloverfield_lane(2016),B1,NaN
1,10_things_I_hate_about_you(1999),B1,NaN
2,A_knights_tale(2001),B2,NaN
3,A_star_is_born(2018),B2,NaN
4,Aladdin(1992),B1,NaN


In [23]:
df2.shape

(241, 3)

In [24]:
df = pd.concat([df, df2], ignore_index=True)

In [25]:
df.head()

,movie,subtitles,label
0,The Walking Dead-S01E01-Days Gone Bye.English,little girl i'm a policeman little girl don't ...,A2
1,The Walking Dead-S01E02-Guts.English,mom right here any luck how do we tell if they...,A2
2,The Walking Dead-S01E03-Tell It To The Frogs.E...,that's right you heard me bitch you got a prob...,A2
3,The Walking Dead-S01E04-Vatos.English,what nothing it's not nothing it's always some...,A2
4,The Walking Dead-S01E05-Wildfire.English,walkie talkie squawks morgan i don't know if y...,A2


In [26]:
df.shape

(404, 3)

<div style="padding: 30px 25px; border: 2px #9966FF solid">

#### вывод по главе 'Загрузка и первичная обработка данных':
- Субтитры загружены и объединены в датасет
- Проведена первичная обработка субтитров:
    - html тэги заменены на пробел
    - комментарии заменены на пробел
    - все что не является буквами заменены на пробел
    - повторяющиеся пробелы заменены на один пробел
    - многоточия заменены на точку
    - удалены все что не ascii символы
    - удалены первый и последний субтитр (обычно это реклама)
    - весь текст переведен в нижний регистр для дальнейшей работы с ним
    

    
</div>

<div style="padding:0px 20px 10px; 
            color:#004346;
            font-size:15px;
            display:fill;
            text-align:center;
            border-radius:20px;
            border: 5px double;
            border-color:#9966FF;
            background-color: #FFFFCC;
            overflow:hidden;
            font-weight:400"> 

# Предобработка и исследовательский анализ данных

</div>


- Названия фильмов не несут информацию о сложности диалогов в нем, поэтому данный признак можно удалить.

In [27]:
# удаление признака `movie`
df = df.drop('movie', axis=1)
# удаление записей с пропусками
df = df.dropna()
# удаление дубликатов
print('дубликаты:', df.duplicated().sum())
df = df.drop_duplicates()
print('дубликаты:', df.duplicated().sum())

дубликаты: 1
дубликаты: 0


In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 162 entries, 0 to 162
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   subtitles  162 non-null    object
 1   label      162 non-null    object
dtypes: object(2)
memory usage: 3.8+ KB


In [29]:
print(df['label'].value_counts())

B2    106
C1     33
B1     17
A2      6
Name: label, dtype: int64


- Cуществует дисбаланс класов

- Небходимо:
используя таблицу [таблицу CEFR/IELS](https://i.postimg.cc/W4YQxZkJ/2023-06-08-15-23-32.png) преобразобать значения в числовой вариант и в последующем решать задачу регрессии

In [30]:
cefr_dict = {'A2': 3.25, #среднее значение крайних значений [3.0 : 3.5]
             'B1': 4.5,  #среднее значение крайних значений [4.0 : 5.0]
             'B2': 6.0, #среднее значение крайних значений [5.5 : 6.5]
             'C1': 7.5,} #среднее значение крайних значений [7.0 : 8.0] 

In [31]:
df['ielts_index'] = df['label'].apply(lambda x: cefr_dict[x]).copy()

<div style="padding:0px 20px 10px; 
            color:#004346;
            font-size:15px;
            display:fill;
            text-align:center;
            border-radius:0px;
            border: 5px double;
            border-color:#9966FF;
            background-color: #FFFFCC;
            overflow:hidden;
            font-weight:400"> 


### Рассчет коофициентов удобочитаемости Флеша-Кинкейда

</div>

<div style="padding: 30px 25px; border: 2px #FF0000 solid">

Тесты на удобочитаемость [Флеша-Кинкейда](https://en.wikipedia.org/wiki/Flesch%E2%80%93Kincaid_readability_tests#Flesch%E2%80%93Kincaid_grade_level) — это тесты на удобочитаемость , предназначенные для определения того, насколько труден для понимания отрывок на английском языке . Есть два теста: Flesch Reading-Ease и Flesch-Kincaid Grade Level. Хотя они используют одни и те же основные меры (длина слова и длина предложения), они имеют разные весовые коэффициенты.

- Flesch Reading-Ease:
$$
FRE = 206.835 - (words/sentences)*1.015 - (syllables/words)*84.6
$$

- Flesch-Kincaid Grade Level:
$$
FKGL = (words/sentences)*0.39 + (syllables/words)*11.8 - 15.59
$$
где:
- words - количество слов в тексте;
- sentences - количество предложений в тексте;
- syllables - количество слогов в тексте
    
</div>

In [32]:
# функции для рассчета коофициентов удобочитаемости:
def statistics(txt):
    total_sentences = len(re.split(r"[.!?]", txt))
    total_words = len(txt.split(' '))
    total_syllables = sum(txt.count(g) for g in 'aeoiu') + txt.count('y')/2
    return total_sentences, total_words, total_syllables
    
def flesch_reading_ease(txt):
    sentences, words, syllables = statistics(txt)
    fres = 206.835 - (words/sentences)*1.015 - (syllables/words)*84.6
    return fres
    
def flesch_kincaid_grade_level(txt):
    sentences, words, syllables = statistics(txt)
    fkgl = (words/sentences)*0.39 + (syllables/words)*11.8 - 15.59
    return fkgl  

In [33]:
df['fres'] = df['subtitles'].apply(flesch_reading_ease)

In [34]:
df['fkgl'] = df['subtitles'].apply(flesch_kincaid_grade_level)

<div style="padding:0px 20px 10px; 
            color:#004346;
            font-size:15px;
            display:fill;
            text-align:center;
            border-radius:0px;
            border: 5px double;
            border-color:#9966FF;
            background-color: #FFFFCC;
            overflow:hidden;
            font-weight:400"> 


### Токенизация и лематизация текста

</div>

- Перед токенизацией текса, необходимо удалить оставшиеся знаки пунктуации и стоп-слова - список слов, которые не влияют на сложность текста, но могут уменьшить точность модели, а также провести Лемматизация и стемминг текста.

<div style="padding: 10px 20px; border: 2px #FF0000 solid">
 
    
-  Токенизация - это процесс разделения предложений на слова-компоненты.
-  Лемматизация - это процесс приведения словоформы к лемме — её нормальной (словарной) форме.
-  стемминг- это процесс нахождения основы слова для заданного исходного слова.
</div>

In [35]:
# Load the English library from SpaCy
nlp = spacy.load("en_core_web_sm")

# Create list of punctuation marks
punctuations = string.punctuation

# Create list of stopwords from spaCy
stopwords = spacy.lang.en.stop_words.STOP_WORDS

In [36]:
# создаем фукцию токенаизер
def spacy_tokenizer(sentence):
    # создаем токен
    tokens = nlp(sentence)
    
    tokens = [word.lemma_.lower().strip() if word.lemma_ != "PROPN" else word.lower_ for word in tokens]
    
    tokens = [word for word in tokens if word not in stopwords and word not in punctuations]
    
    return tokens

In [37]:
tfidf_vector = TfidfVectorizer(tokenizer = spacy_tokenizer,max_features=25)

<div style="padding:0px 20px 10px; 
            color:#004346;
            font-size:15px;
            display:fill;
            text-align:center;
            border-radius:0px;
            border: 5px double;
            border-color:#9966FF;
            background-color: #FFFFCC;
            overflow:hidden;
            font-weight:400"> 
    
### Рассчет долей слов относительно индекса CEFR

</div>

In [38]:
# cоздание пустого датафрейма для хранения данных
df_info = pd.DataFrame(columns=['a1', 'a2', 'b1', 'b2', 'c1'])

In [39]:
data_info = []
# функция рассчета доли слов по уровням сложности
def oxford_cefr(x):
    a1 = sum(1 for i in x if i in a1_list)
    a2 = sum(1 for i in x if i in a2_list)
    b1 = sum(1 for i in x if i in b1_list)
    b2 = sum(1 for i in x if i in b2_list)
    c1 = sum(1 for i in x if i in c1_list)
    count_word = a1+a2+b1+b2+c1
    a1 = a1/count_word
    a2 = a2/count_word
    b1 = b1/count_word
    b2 = b2/count_word
    c1 = c1/count_word
    data_info.append({'a1':a1, 'a2':a2, 'b1':b1, 'b2': b2, 'c1':c1})

In [40]:
%%time
df['subtitles'].apply(oxford_cefr)

CPU times: total: 9min 50s
Wall time: 9min 52s


0      None
1      None
2      None
3      None
4      None
       ... 
158    None
159    None
160    None
161    None
162    None
Name: subtitles, Length: 162, dtype: object

In [41]:
# объединяем все записи в датасете с помощью функции concat
df_info = pd.concat([df_info, pd.DataFrame(data_info)], ignore_index=True)
df = pd.concat([df, df_info], axis=1).copy()
# удаление записей с пропусками
df = df.dropna()

In [42]:
df.head()

,subtitles,label,ielts_index,fres,fkgl,a1,a2,b1,b2,c1
0,little girl i'm a policeman little girl don't ...,A2,3.25,-3069.338469,1210.751560,0.599267,0.034799,0.331136,0.034799,0.0
1,mom right here any luck how do we tell if they...,A2,3.25,-3036.904170,1198.774790,0.591381,0.040785,0.327049,0.040785,0.0
2,that's right you heard me bitch you got a prob...,A2,3.25,-3952.166828,1551.262784,0.608133,0.034036,0.323795,0.034036,0.0
3,what nothing it's not nothing it's always some...,A2,3.25,-3663.925591,1440.008546,0.600251,0.032268,0.335213,0.032268,0.0
4,walkie talkie squawks morgan i don't know if y...,A2,3.25,-2952.000335,1166.312895,0.608340,0.037975,0.315711,0.037975,0.0


<div style="padding: 30px 25px; border: 2px #9966FF solid">

#### вывод по главе 'Предобработка и исследовательский анализ данных':
    
- Изучена общая информация.
- Обработаны дубликаты и пропуски в данных.
- Выявлен дисбаланс классов.
- Удалены стоп-слова.
- Проведена токенизация и лематизация текста.
- Рассчитаны дополнительные признаки:
    - Flesch Reading-Ease
    - Flesch-Kincaid Grade Level
    - Доли слов относительно индекса CEFR
    
</div>

<div style="padding:0px 20px 10px; 
            color:#004346;
            font-size:15px;
            display:fill;
            text-align:center;
            border-radius:20px;
            border: 5px double;
            border-color:#9966FF;
            background-color: #FFFFCC;
            overflow:hidden;
            font-weight:400"> 

# Разработка модели ML
</div>

- Для обучения моделей машинного обучения по прежнему не хватает записей, но так как субтитры в большенстве своем достаточно длинные, мы в праве разделить их по определенному количеству слов, тем самым увеличив количество записей. При этом рассчитанные ранее статистические данные следует оставить без изменений, тк они были рассчитаны на всем тексте и обладают большей точностью, по сравнению с теми, которые мы можем рассчитать на части данных.
   - выберем количество: 100 слов 
- Некоторые записи могут содержать меньшее число слов, их следует исключить.

In [43]:
# определение количества слов в субтитрах
df['len'] = df['subtitles'].apply(lambda x: len(x))

In [44]:
df = df[df['len']>=100]

In [45]:
# cоздание пустого датафрейма для хранения данных
df_div = pd.DataFrame(columns=['subtitles','a1', 'a2', 'b1', 'b2', 'c1', 'fres', 'fkgl','label', 'ielts_index'])

In [46]:
# функция для разделения субтитров
len_div = 100 #количество слов в подвыборке, для увеличения наблюдений
data = []
def text_division(x):
    for i in range(len(x['subtitles'])//len_div):
        data.append({'subtitles': ' '.join(x['subtitles'][i*len_div:(i+1)*len_div+1]), 
                     'a1': x['a1'],
                     'a2': x['a2'],
                     'b1': x['b1'],
                     'b2': x['b2'],
                     'c1': x['c1'],
                     'fres': x['fres'],
                     'fkgl': x['fkgl'],
                     'label': x['label'],
                     'ielts_index': x['ielts_index']})                 

In [47]:
%%time
df.apply(text_division, axis=1)

CPU times: total: 2.5 s
Wall time: 2.5 s


0      None
1      None
2      None
3      None
4      None
       ... 
157    None
158    None
159    None
160    None
161    None
Length: 161, dtype: object

In [48]:
# объединяем все записи в датасете с помощью функции concat
df = pd.concat([df_div, pd.DataFrame(data)], ignore_index=True)

<div style="padding:0px 20px 10px; 
            color:#004346;
            font-size:15px;
            display:fill;
            text-align:center;
            border-radius:20px;
            border: 5px double;
            border-color:#9966FF;
            background-color: #FFFFCC;
            overflow:hidden;
            font-weight:400"> 
    
### TF-IDF векторизация

</div>


<div style="padding: 10px 20px; border: 2px #FF0000 solid">
 
- Векторизация текста - это процесс преобразования текстовых данных в числовые векторы, которые могут быть использованы в алгоритмах машинного обучения или анализа данных.
    
    
- TF-IDF -  это метод который присваивает каждому слову в тексте числовое значение, основанное на его частоте встречаемости в документе (Term Frequency) и обратной частоте встречаемости в корпусе документов (Inverse Document Frequency).Результатом векторизации текста с использованием TF-IDF является числовое представление текста, где каждое слово представлено числовым значением, отражающим его важность в контексте данного текста.
    
<div>

- Для обучения модели, необходима векторизация текстовой информации.
- Так же необходимо ограничить размерность матрицы, т.к. в случае сохранения всех параметров, потребуется много времени для реализации данного алгоритма обучения.

In [50]:
# признаки для обучения
X = ['subtitles', 'a1', 'a2', 'b1', 'b2', 'c1', 'fres', 'fkgl']
# целевой признак
y = ['ielts_index']

# выборки для подбора гиперпараметров CatBoost
features_train, features_valid, target_train, target_valid = train_test_split(df[X], df[y], test_size=0.2,
                                                                              random_state=RANDOM_SEED)
features_train, features_test, target_train, target_test = train_test_split(features_train, target_train,
                                                                            test_size=0.1, random_state=RANDOM_SEED)
print(f'размер тренировочной выборки: {features_train.shape}, {target_train.shape}' )
print(f'размер валидационной выборки:{features_valid.shape}, {target_valid.shape}')
print(f'размер тестовой выборки:{features_test.shape}, {target_test.shape}')

размер тренировочной выборки: (34696, 8), (34696, 1)
размер валидационной выборки:(9639, 8), (9639, 1)
размер тестовой выборки:(3856, 8), (3856, 1)


In [51]:
%%time
features_train_vectorized = tfidf_vector.fit_transform(features_train['subtitles'])
features_test_vectorized = tfidf_vector.transform(features_test['subtitles'])
# Вывод размерности матрицы TF-IDF
print("Размерность обучающей матрицы TF-IDF:", features_train_vectorized.shape)
print("Размерность тестовой матрицы TF-IDF:", features_test_vectorized.shape)

# Преобразование векторизованного столбца 'subtitles' в массив NumPy
features_train_vectorized = features_train_vectorized.toarray()
features_test_vectorized = features_test_vectorized.toarray()

# Объединение данных в один набор
features_train_combined = np.hstack((features_train_vectorized, features_train.drop('subtitles', axis=1)))
features_test_combined = np.hstack((features_test_vectorized, features_test.drop('subtitles', axis=1)))

Размерность обучающей матрицы TF-IDF: (34696, 24)
Размерность тестовой матрицы TF-IDF: (3856, 24)
CPU times: total: 20min 31s
Wall time: 20min 32s


<div style="padding: 30px 25px; border: 2px #6495ed solid">

#### промежуточный вывод:
- Увеличен размер выборки за счет разделения субтитров на несколько записей.
- Датафрейм разделен на обучающую и тестовую выборки.
    
</div>

<div style="padding:0px 20px 10px; 
            color:#004346;
            font-size:15px;
            display:fill;
            text-align:center;
            border-radius:20px;
            border: 5px double;
            border-color:#9966FF;
            background-color: #FFFFCC;
            overflow:hidden;
            font-weight:400"> 
    
# Обучение моделей
<div>

<div style="padding:0px 20px 10px; 
            color:#004346;
            font-size:15px;
            display:fill;
            text-align:center;
            border-radius:0px;
            border: 5px double;
            border-color:#9966FF;
            background-color: #FFFFCC;
            overflow:hidden;
            font-weight:400"> 
    
### RandomForestRegressor

</div>


In [52]:
# гиперпараметры модели
param_grid = {
    'n_estimators': [100, 150],
    'max_depth': [5, 7],
    'min_samples_split': [2, 3]
}
# параметры модели
model_RF = RandomForestRegressor(random_state=RANDOM_SEED)
# объект метрики MAE
scorer = make_scorer(mean_absolute_error)
# параметры RandomizedSearchCV
random_search_RF = RandomizedSearchCV(estimator=model_RF,
                                      param_distributions=param_grid,
                                      n_iter=100,
                                      cv=3,
                                      verbose=True,
                                      random_state=RANDOM_SEED,
                                      scoring=scorer,
                                      n_jobs=-1)

In [53]:
%%time
# обучение модели
random_search_RF.fit(features_train_combined, target_train.values.ravel())
# сохраним лучшую модель
best_model_RF = random_search_RF.best_estimator_
# сохраним лучшее значение метрики
final_metrics_RF = random_search_RF.best_score_
# вывод результатов
print(f'Оптимальные гиперпараметры:\n{random_search_RF.best_params_}\n mae: {final_metrics_RF}')

Fitting 3 folds for each of 8 candidates, totalling 24 fits
Оптимальные гиперпараметры:
{'n_estimators': 150, 'min_samples_split': 2, 'max_depth': 5}
 mae: 0.41439941231419164
CPU times: total: 1min 8s
Wall time: 5min 12s


<div style="padding: 30px 25px; border: 2px #6495ed solid">

#### промежуточный вывод:
- Оптимальные гитерпараметры для алгоритма RandomForest:
    - 'n_estimators': 150
    - 'min_samples_split': 2
    - 'max_depth': 5
---
- Метрика качества MAE на обучающих данных при кроссвалидации составляет: 0.41
    
</div>

<div style="padding:0px 20px 10px; 
            color:#004346;
            font-size:15px;
            display:fill;
            text-align:center;
            border-radius:0px;
            border: 5px double;
            border-color:#9966FF;
            background-color: #FFFFCC;
            overflow:hidden;
            font-weight:400">
    
### CatBoost
  
<div>    

<div style="padding: 10px 20px; border: 2px #FF0000 solid">
    
- Градиентный бустинг (Gradient Boosting Machine - GBM) – это метод Машинного обучения (ML) для задач Регрессии (Regression) и Классификации (Classification), который создает прогнозирующую Модель (Model) в форме Ансамбля (Ensemble) слабых алгоритмов прогнозирования, обычно Деревьев решений (Decision Tree).
- Ансамбль – это просто набор ML моделей, которые собираются вместе с помощью, например, среднего значения всех прогнозов, чтобы дать окончательный.
- CatBoost - это градиентный бустинговый алгоритм, разработанный компанией Yandex. Он является мощным инструментом для задач классификации и регрессии, который обладает рядом преимуществ и особенностей.
    
<div>

In [79]:
# тестовые признаки 
text_features = ['subtitles']
# pool
train_pool = Pool(features_train,
                  target_train,
                  text_features=text_features)

test_pool = Pool(features_test,
                 text_features=text_features)
# целевой признак тестовой выборки
target_test_cb = target_test

<div style="padding:1px 20px 10px; 
            color:#004346;
            font-size:15px;
            display:fill;
            text-align:center;
            border-radius:0px;
            border: 5px double;
            border-color:#9966FF;
            background-color: #FFFFCC;
            overflow:hidden;
            font-weight:400"> 
                
### optuna-подбор оптимальных гиперпараметров
<div>

In [55]:
# гиперпараметры optuna
def objective(trial):
    param = {}
    param['learning_rate'] = trial.suggest_discrete_uniform("learning_rate", 0.01, 0.02, 0.001)
    param['depth'] = trial.suggest_int('depth', 9, 15)
    param['l2_leaf_reg'] = trial.suggest_discrete_uniform('l2_leaf_reg', 1.0, 5.5, 0.5)
    param['min_child_samples'] = trial.suggest_categorical('min_child_samples', [1, 4, 8, 16, 32])
    param['grow_policy'] = 'Depthwise'
    param['iterations'] = 1000
    param['use_best_model'] = True
    param['eval_metric'] = 'MAE'
    param['od_type'] = 'iter'
    param['od_wait'] = 20
    param['random_state'] = RANDOM_SEED
    param['logging_level'] = 'Silent'
    param['text_features'] = text_features
    
    regressor = CatBoostRegressor(**param)

    regressor.fit(features_train.copy(), target_train.copy(),
                  eval_set=[(features_test.copy(), target_test.copy())],
                  early_stopping_rounds=EARLY_STOPPING_ROUND)
    loss = mean_absolute_error(target_valid, regressor.predict(features_valid.copy()))
    return loss

In [56]:
%%time
study = optuna.create_study(study_name=f'catboost-seed{RANDOM_SEED}')
study.optimize(objective, n_trials=1000, n_jobs=-1, timeout=2400)

[I 2023-07-29 18:33:21,903] A new study created in memory with name: catboost-seed3826
[I 2023-07-29 20:03:04,494] Trial 3 finished with value: 0.0041476218377971925 and parameters: {'learning_rate': 0.02, 'depth': 12, 'l2_leaf_reg': 1.5, 'min_child_samples': 32}. Best is trial 3 with value: 0.0041476218377971925.
[I 2023-07-29 20:50:34,165] Trial 1 finished with value: 0.005751127439628427 and parameters: {'learning_rate': 0.02, 'depth': 14, 'l2_leaf_reg': 3.0, 'min_child_samples': 32}. Best is trial 3 with value: 0.0041476218377971925.
[I 2023-07-29 21:05:49,800] Trial 2 finished with value: 0.005014450154523328 and parameters: {'learning_rate': 0.019000000000000003, 'depth': 14, 'l2_leaf_reg': 2.0, 'min_child_samples': 32}. Best is trial 3 with value: 0.0041476218377971925.
[I 2023-07-29 22:44:21,171] Trial 4 finished with value: 0.00510047017937372 and parameters: {'learning_rate': 0.018000000000000002, 'depth': 15, 'l2_leaf_reg': 1.0, 'min_child_samples': 16}. Best is trial 3 with

CPU times: total: 7h 59min 15s
Wall time: 4h 27min 33s


In [57]:
print(f' лучшая метрика: \n{study.best_value}\n оптимальные гиперпараметры: {study.best_params}')

 лучшая метрика: 
0.0041476218377971925
 оптимальные гиперпараметры: {'learning_rate': 0.02, 'depth': 12, 'l2_leaf_reg': 1.5, 'min_child_samples': 32}


<div style="padding: 30px 25px; border: 2px #6495ed solid">

#### промежуточный вывод:
- Оптимальные гитерпараметры для алгоритма CatBoost, при использовании алгоритма optuna:
    - 'learning_rate': 0.02
    - 'depth': 12
    - 'l2_leaf_reg': 1.5
    - 'min_child_samples': 32
    
- Метрика качества MAE на обучающих данных при кроссвалидации составляет: 0.004
    ____________
    
- Для дальнейшего обучения выбираем алгоритм CatBoost, показывающий лучшую метрику качества. Однако, часть параметров не будет использоваться далее, что увеличит скорость обучения и предсказаний. В дальнейшем, при необходимости более точных данных, можно их можно будет использовать в модели.
    
</div>

In [80]:
# гиперпараметры модели
parameters = {'verbose': VERBOSE,
              'text_features': ['subtitles'],
              'eval_metric': 'MAE',
              'iterations': 1000,
              'learning_rate': 0.02,
              'random_seed':RANDOM_SEED,
              'early_stopping_rounds': 30
             }
# параметры модели
regressor = CatBoostRegressor(**parameters)

In [81]:
%%time
# обучение модели
regressor.fit(train_pool)

0:	learn: 0.5867878	total: 33.3ms	remaining: 33.3s
100:	learn: 0.3589777	total: 3.27s	remaining: 29.2s
200:	learn: 0.2528778	total: 6.32s	remaining: 25.1s
300:	learn: 0.1883388	total: 9.35s	remaining: 21.7s
400:	learn: 0.1423854	total: 12.7s	remaining: 18.9s
500:	learn: 0.1094007	total: 16s	remaining: 16s
600:	learn: 0.0832777	total: 19.1s	remaining: 12.7s
700:	learn: 0.0647134	total: 22.3s	remaining: 9.51s
800:	learn: 0.0497025	total: 25.7s	remaining: 6.38s
900:	learn: 0.0380284	total: 29.3s	remaining: 3.22s
999:	learn: 0.0303971	total: 32.6s	remaining: 0us


<div style="padding:1px 20px 10px; 
            color:#004346;
            font-size:15px;
            display:fill;
            text-align:center;
            border-radius:0px;
            border: 5px double;
            border-color:#9966FF;
            background-color: #FFFFCC;
            overflow:hidden;
            font-weight:400"> 

## Проверка модели на тестовой выборке
    
<div>

In [82]:
# предказание данных
predict = regressor.predict(test_pool)
mae = mean_absolute_error(target_test_cb, predict)
# вывод результатов
print("Средняя абсолютная ошибка (MAE):", mae)

Средняя абсолютная ошибка (MAE): 0.030059597477530956


In [83]:
# сохранение итоговой модели
regressor.save_model('catboost_model')

<div style="padding:0px 20px 10px; 
            color:#004346;
            font-size:15px;
            display:fill;
            text-align:center;
            border-radius:20px;
            border: 5px double;
            border-color:#9966FF;
            background-color: #FFFFCC;
            overflow:hidden;
            font-weight:400"> 

# Вывод
    
<div>

- В ходе проведения исследования была выполнена предобработка данных, включающая очистку и лемматизацию текстовых признаков. Для векторизации текста был применен метод TF-IDF, который позволяет представить тексты в виде числовых признаков, учитывающих важность каждого термина в документе.

- Для настройки гиперпараметров модели CatBoost был использован фреймворк Optuna. Optuna провел исследование пространства гиперпараметров, оценивая производительность модели с различными комбинациями параметров. Целевая метрика, в данном случае MAE, была определена для оптимизации. В результате подбора гиперпараметров были получены оптимальные значения, которые позволяют достичь наилучшей производительности модели CatBoost на данной задаче и составиляют 0.03 на тестовой выборке.

- Полученная модель будет использоваться в разработке сервиса, предназначенного для определения уровня сложности английского языка в фильмах. Этот сервис будет доступен через платформу Streamlit, что позволит пользователям оценивать и анализировать сложность субтитров и основываться на предсказанных значениях индекса IELTS или CEFR